In [8]:
# Import necessary libraries
import pandas as pd
from datetime import datetime
import os
import glob
import calendar

# Function to check if a text does not contain the string "order"
def not_contains_order(text):
    if isinstance(text, str):
        return 'order' not in text.lower()
    return True

# Function to convert a string to a float
def convert_to_float(x):
    try:
        return float(x.replace('CA$', '').replace(',', ''))
    except ValueError:
        return None

# Main function to process and analyze operating costs
def Operating_Costs ():
    
    # Specify the directory where the CSV files are located
    path = 'C:\\Users\\User\\Python\\Projects\\Etsy Operating Costs\\Etsy Data'
    # Specify the directory where the output files should be saved
    output_path = 'C:\\Users\\User\\Python\\Projects\\Etsy Operating Costs\\Scraped Data'
    extension = 'csv'
    os.chdir(path)
    result = glob.glob('*.{}'.format(extension))
    #print(result)

    # Create an empty DataFrame to store the merged data
    merged_data = pd.DataFrame()

    # Process each CSV file in the specified directory
    for entry in result:
        if 'etsy_statement' in entry:
            # Load the CSV file into a DataFrame
            order_df = pd.read_csv(entry)

            # Extract the date information from the first row of the DataFrame
            date = order_df['Date'].iloc[0]
            date = date.replace(',', '')
            date = datetime.strptime(date, '%d %B %Y')
            month = date.month
            year = date.year
            # Convert the values in the "Net" column to a float
            order_df['Net'] = order_df['Net'].apply(convert_to_float)

            # Filter out any entries with an empty value in the "Net" column
            order_df.dropna(subset=['Net'], inplace=True)

            # Filter the DataFrame to exclude any rows that contain the string "order" in the "Info" column or "Sale" in the "Type" column
            filtered_order_df = order_df[order_df['Info'].apply(not_contains_order)]
            filtered_order_df = filtered_order_df[filtered_order_df['Type'] != 'Sale']

            # Group the DataFrame by unique entries in the "Title" column and aggregate the "Net" column by summing the values
            grouped_df = filtered_order_df.groupby('Title', as_index=False).agg({'Net': 'sum'})

            # Rename the "Title" column to "Expense Type"
            grouped_df.rename(columns={'Title': 'Expense Type'}, inplace=True)

            # Merge the specified 'Expense Type' entries
            merge_dict = {
                'Tax: Etsy Ads': 'Etsy Ads',
                'Tax: transaction credit': 'Tax: transaction',
                'Tax: renew sold auto credit': 'Tax: renew',
                'Tax: renew sold credit': 'Tax: renew',
                'Tax: renew sold': 'Tax: renew',
                'Tax: private listing': 'Tax: listing',
                'Credit for listing fee ($0.20 USD)': 'Listing fee ($0.20 USD)',
                'Tax: listing' : 'Listing fee ($0.20 USD)',
                'Credit for Etsy Ads fee' : 'Etsy Ads',
                'Credit for listing fee' : 'Listing fee ($0.20 USD)',
                'Tax: Etsy Plus subscription' : 'Etsy Plus subscription fee (10.00 USD)',
                'Tax: auto-renew sold ': 'Tax: renew',
                'Tax: shipping_transaction' : 'Tax: transaction',
            }
            grouped_df['Expense Type'] = grouped_df['Expense Type'].replace(merge_dict)
            grouped_df = grouped_df.groupby('Expense Type', as_index=False).agg({'Net': 'sum'})
            
            # Remove any expense types that include "Charge for refund"
            grouped_df = grouped_df[~grouped_df['Expense Type'].str.contains('Charge for refund')]

            # Add a 'month' column to the grouped_df DataFrame with the month name
            month_name = calendar.month_name[month]
            grouped_df['month'] = month_name

            # Append the grouped_df DataFrame to the merged_data DataFrame
            merged_data = pd.concat([merged_data, grouped_df], ignore_index=True)

            # Save the grouped data to a CSV file
            output_file = 'Operating_Costs_{}_{}.csv'.format(year, month)
            output_file_path = os.path.join(output_path, output_file)
            grouped_df.to_csv(output_file_path, index=False)

    # Save the merged data to a single CSV file
    merged_output_file = os.path.join(output_path, 'Merged_Operating_Costs.csv')
    merged_data.to_csv(merged_output_file, index=False)


# Call the main function
Operating_Costs()
